In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Clone project from github
#! rm -r advanced-DL-project
! git clone "https://github.com/RebwarBajallan95/advanced-DL-project"
! cd advanced-DL-project

import os
os.chdir("./advanced-DL-project")

fatal: destination path 'advanced-DL-project' already exists and is not an empty directory.


In [3]:
import torch
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
from mimo_resnet28_10 import mimo_wide_resnet28

%load_ext autoreload
%autoreload 2

torch.backends.cudnn.benchmark = True

# set seed
torch.manual_seed(0)
# set torch.device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Print to see if you have GPU available, if no GPU => change colab runtime
print(f'GPU: {torch.cuda.get_device_name(0)}')

GPU: Tesla T4


In [13]:
cifar10_mean = (0.4914, 0.4822, 0.4465) 
cifar10_std = (0.2470, 0.2435, 0.2616)

# Image preprocessing modules
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(cifar10_mean, cifar10_std)])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(
                                        trainset, 
                                        batch_size=batch_size,
                                        shuffle=True, 
                                        pin_memory=True,
                                        drop_last=True, 
                                        
                                    )

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
                                       
testloader = torch.utils.data.DataLoader(
                                    testset, 
                                    batch_size=batch_size,
                                    shuffle=False,
                                    pin_memory=True,
                                )

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [14]:
import numpy as np
from mimo_resnet28_10 import mimo_wide_resnet28

%load_ext autoreload
%autoreload 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = mimo_wide_resnet28(
    input_shape=(3, 3, 32, 32), # NOTE: PyTorch expects (n_samples, channels, height, width)
    width_multiplier=10,
    num_classes=10,
    ensemble_size=3,
    batch_repitition=4
).to(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
torch.cuda.empty_cache()
model.fit(
        trainloader, 
        testloader, 
        epochs=250, 
        trainset_size=len(trainset), 
        batch_size=batch_size
    )

Learning rate:  [0.1]
Epoch:  0


  0%|          | 0/390 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 80.00 MiB (GPU 0; 14.76 GiB total capacity; 13.13 GiB already allocated; 75.75 MiB free; 13.38 GiB reserved in total by PyTorch)

In [11]:
import gc
  
gc.collect()
torch.cuda.empty_cache()